In [1]:
import platform

platform.python_version()

'3.9.16'

In [2]:
import numpy as np

In [3]:
np.__version__

'1.24.3'

In [4]:
import pandas as pd

In [5]:
pd.__version__

'2.0.1'

In [6]:
import polars as pl

In [7]:
pl.__version__

'0.19.3'

In [8]:
import pyarrow

In [21]:
import time

In [9]:
num_rows = 5000
rng = np.random.default_rng(seed=7)

In [10]:
buildings_data = {
      "sqft": rng.exponential(scale=1000, size=num_rows),
      "year": rng.integers(low=1995, high=2023, size=num_rows),
      "building_type": rng.choice(["A", "B", "C"], size=num_rows),
}

In [11]:
buildings = pl.DataFrame(buildings_data)

In [12]:
buildings

sqft,year,building_type
f64,i64,str
707.529256,1996,"""C"""
1025.203348,2020,"""C"""
568.548657,2012,"""A"""
895.109864,2000,"""A"""
206.532754,2011,"""A"""
3383.637351,2018,"""B"""
9.753627,2007,"""A"""
2809.215763,2004,"""B"""
575.332756,2021,"""A"""


In [13]:
buildings.head()

sqft,year,building_type
f64,i64,str
707.529256,1996,"""C"""
1025.203348,2020,"""C"""
568.548657,2012,"""A"""
895.109864,2000,"""A"""
206.532754,2011,"""A"""


In [14]:
buildings.shape

(5000, 3)

In [15]:
type(buildings)

polars.dataframe.frame.DataFrame

In [16]:
help(buildings.to_pandas)

Help on method to_pandas in module polars.dataframe.frame:

to_pandas(*args: 'Any', use_pyarrow_extension_array: 'bool' = False, **kwargs: 'Any') -> 'pd.DataFrame' method of polars.dataframe.frame.DataFrame instance
    Cast to a pandas DataFrame.
    
    This requires that :mod:`pandas` and :mod:`pyarrow` are installed.
    This operation clones data, unless `use_pyarrow_extension_array=True`.
    
    Parameters
    ----------
    use_pyarrow_extension_array
        Use PyArrow backed-extension arrays instead of numpy arrays for each column
        of the pandas DataFrame; this allows zero copy operations and preservation
        of null values. Subsequent operations on the resulting pandas DataFrame may
        trigger conversion to NumPy arrays if that operation is not supported by
        pyarrow compute functions.
    **kwargs
        Arguments will be sent to :meth:`pyarrow.Table.to_pandas`.
    
    Returns
    -------
    :class:`pandas.DataFrame`
    
    Examples
    ------

In [19]:
df = buildings.to_pandas()

In [20]:
type(df)

pandas.core.frame.DataFrame

In [23]:
train_pd=pd.read_parquet('./archive/train.parquet') #Pandas dataframe

train_pl=pl.read_parquet('./archive/train.parquet') #Polars dataframe

In [35]:
train_pd.columns.str.contains('num') 

array([False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [36]:
nums = train_pd.columns[train_pd.columns.str.contains('num')] 

In [37]:
nums

Index(['num_7', 'num_8', 'num_9', 'num_10', 'num_11', 'num_12', 'num_13',
       'num_14', 'num_15'],
      dtype='object')

In [38]:
cats = train_pd.columns[train_pd.columns.str.contains('cat')] 

In [39]:
cats

Index(['cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6'], dtype='object')

In [40]:
# pandas query
train_pd[nums].agg(['min','max','mean','median','std'])


,num_7,num_8,num_9,num_10,num_11,num_12,num_13,num_14,num_15
min,3.447814e-09,6.758090e-08,3.387718e-09,2.832512e-09,7.106527e-08,9.356119e-09,1.062366e-08,3.810326e-08,4.786359e-08
max,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
mean,4.999846e-01,4.999866e-01,5.000210e-01,5.001010e-01,4.999404e-01,5.000311e-01,5.000412e-01,4.999558e-01,5.000969e-01
median,5.000017e-01,4.999354e-01,5.000797e-01,5.001719e-01,4.999099e-01,5.000381e-01,5.000839e-01,4.999062e-01,5.001162e-01
std,2.886756e-01,2.886867e-01,2.887038e-01,2.887107e-01,2.887081e-01,2.886917e-01,2.886958e-01,2.886739e-01,2.886854e-01


In [42]:
train_pd[cats].agg(['nunique'])

,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6
nunique,4,64,31,77,71,67


In [45]:
pl.col(nums[0])

In [48]:
# Polars query
train_pl.with_columns([
    pl.col(nums[0]).min().suffix('_min'),
    pl.col(nums[0]).max().suffix('_max'),
    pl.col(nums[0]).mean().suffix('_mean'),
    pl.col(nums[0]).median().suffix('_median'),
    pl.col(nums[0]).std().suffix('_std'),
    pl.col(cats[0]).n_unique().suffix('_unique'),
])

user,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,num_7,num_8,num_9,num_10,num_11,num_12,num_13,num_14,num_15,num_7_min,num_7_max,num_7_mean,num_7_median,num_7_std,cat_1_unique
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u32
37460,2,60,8,73,17,61,0.119154,0.79259,0.691714,0.413575,0.56896,0.724723,0.667784,0.621327,0.366504,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
6199,1,30,14,31,65,49,0.871286,0.864431,0.406232,0.685368,0.425132,0.184816,0.30766,0.497363,0.740807,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
67094,2,17,7,18,8,29,0.249553,0.950156,0.081369,0.563896,0.451576,0.77827,0.694566,0.807619,0.916968,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
97776,3,60,16,28,32,43,0.545136,0.463736,0.377657,0.728032,0.537577,0.943349,0.194909,0.684796,0.422892,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
18048,2,42,22,49,53,67,0.606616,0.757299,0.910864,0.187922,0.935874,0.810835,0.152005,0.035744,0.941222,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
82555,4,31,12,7,36,28,0.516963,0.171398,0.67967,0.396448,0.490542,0.670266,0.416819,0.027584,0.632403,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
5500,2,22,27,32,51,1,0.968847,0.402792,0.402879,0.128054,0.856646,0.38113,0.240961,0.475047,0.658598,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
84103,4,41,13,29,2,61,0.034976,0.811376,0.074072,0.797366,0.41293,0.584796,0.452564,0.972065,0.717349,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
88197,3,34,3,9,33,57,0.608963,0.194878,0.783312,0.732643,0.014411,0.021338,0.906797,0.181182,0.336104,3.4478e-9,1.0,0.499985,0.500002,0.288676,4
